In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('mimic-iii-clinical-database-demo-1.4/LABEVENTS.csv')
df.head()

,row_id,subject_id,hadm_id,itemid,charttime,value,valuenum,valueuom,flag
0,6244563,10006,NaN,50868,2164-09-24 20:21:00,19,19.0,mEq/L,NaN
1,6244564,10006,NaN,50882,2164-09-24 20:21:00,27,27.0,mEq/L,NaN
2,6244565,10006,NaN,50893,2164-09-24 20:21:00,10.0,10.0,mg/dL,NaN
3,6244566,10006,NaN,50902,2164-09-24 20:21:00,97,97.0,mEq/L,NaN
4,6244567,10006,NaN,50912,2164-09-24 20:21:00,7.0,7.0,mg/dL,abnormal


In [4]:
df = df[['subject_id', 'hadm_id', 'itemid', 'valuenum']]
df.head

<bound method NDFrame.head of        subject_id   hadm_id  itemid  valuenum
0           10006       NaN   50868     19.00
1           10006       NaN   50882     27.00
2           10006       NaN   50893     10.00
3           10006       NaN   50902     97.00
4           10006       NaN   50912      7.00
...           ...       ...     ...       ...
76069       44228  103379.0   51250     88.00
76070       44228  103379.0   51265    595.00
76071       44228  103379.0   51277     14.50
76072       44228  103379.0   51279      2.76
76073       44228  103379.0   51301     16.90

[76074 rows x 4 columns]>

In [5]:
pivot_df = df.pivot_table(
    index=['subject_id', 'hadm_id'],  # Group by these identifiers
    columns='itemid',                 # Convert unique itemids into columns
    values='valuenum',                # Populate columns with valuenum
    aggfunc='first'                   # Handle duplicates (use 'first', 'mean', etc.)
).reset_index()                       # Flatten the index

pivot_df.columns.name = None

pivot_df

,subject_id,hadm_id,50801,50802,50803,50804,50805,50806,50808,50809,...,51482,51484,51491,51492,51493,51498,51501,51507,51514,51516
0,10006,142345.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,8.0,30.0,NaN,1.012,NaN,NaN,NaN,NaN
1,10011,105331.0,NaN,0.0,NaN,23.0,NaN,NaN,NaN,NaN,...,NaN,NaN,6.5,100.0,NaN,1.020,NaN,NaN,0.2,2.0
2,10013,165520.0,NaN,2.0,NaN,32.0,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,1.010,NaN,NaN,0.2,NaN
3,10017,199207.0,NaN,5.0,NaN,32.0,NaN,NaN,NaN,NaN,...,NaN,150.0,5.0,NaN,NaN,1.024,NaN,NaN,NaN,NaN
4,10019,177759.0,588.0,-9.0,NaN,19.0,NaN,NaN,0.93,146.0,...,NaN,NaN,5.0,30.0,NaN,1.006,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,44083,198330.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
125,44154,174245.0,581.0,-3.0,24.0,24.0,NaN,107.0,0.98,164.0,...,NaN,NaN,5.0,30.0,NaN,1.017,NaN,NaN,NaN,NaN
126,44212,163189.0,637.0,-7.0,NaN,20.0,NaN,NaN,1.07,117.0,...,NaN,NaN,5.0,NaN,NaN,1.020,NaN,NaN,0.2,NaN
127,44222,192189.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,15.0,NaN,5.0,30.0,NaN,1.016,NaN,NaN,NaN,1.0


In [6]:
pivot_df.to_csv('sample_data/labevents_pivoted.csv', index=False)

In [7]:
na = pivot_df.isnull().sum()

d = dict(na)

for i in d:
    if (d[i]/129) > 0.5:
        pivot_df.drop(i, axis=1, inplace=True)

pivot_df.head()

,subject_id,hadm_id,50802,50804,50813,50818,50820,50821,50822,50861,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,10006,142345.0,NaN,NaN,4.4,NaN,NaN,NaN,NaN,9.0,...,5.0,86.6,116.0,21.2,32.7,20.0,4.46,7.8,8.0,1.012
1,10011,105331.0,0.0,23.0,NaN,31.0,7.47,119.0,NaN,254.0,...,7.0,85.0,299.0,35.4,66.0,18.9,4.07,10.6,6.5,1.020
2,10013,165520.0,2.0,32.0,NaN,63.0,7.30,60.0,NaN,29.0,...,7.0,74.0,220.0,14.6,30.4,14.2,3.48,16.2,5.0,1.010
3,10017,199207.0,5.0,32.0,NaN,47.0,7.42,138.0,NaN,12.0,...,6.0,75.0,399.0,11.3,19.6,12.6,4.84,30.5,5.0,1.024
4,10019,177759.0,-9.0,19.0,6.8,46.0,7.20,123.0,NaN,176.0,...,NaN,NaN,68.0,32.2,150.0,18.0,3.73,3.7,5.0,1.006


In [8]:
pivot_df.fillna(pivot_df.mean(numeric_only=True), inplace=True)

pivot_df

,subject_id,hadm_id,50802,50804,50813,50818,50820,50821,50822,50861,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,10006,142345.0,-0.703297,25.681319,4.40000,44.197802,7.362632,167.428571,4.107692,9.00000,...,5.000000,86.600000,116.0,21.2,32.7,20.0,4.46,7.8,8.000000,1.012000
1,10011,105331.0,0.000000,23.000000,2.50297,31.000000,7.470000,119.000000,4.107692,254.00000,...,7.000000,85.000000,299.0,35.4,66.0,18.9,4.07,10.6,6.500000,1.020000
2,10013,165520.0,2.000000,32.000000,2.50297,63.000000,7.300000,60.000000,4.107692,29.00000,...,7.000000,74.000000,220.0,14.6,30.4,14.2,3.48,16.2,5.000000,1.010000
3,10017,199207.0,5.000000,32.000000,2.50297,47.000000,7.420000,138.000000,4.107692,12.00000,...,6.000000,75.000000,399.0,11.3,19.6,12.6,4.84,30.5,5.000000,1.024000
4,10019,177759.0,-9.000000,19.000000,6.80000,46.000000,7.200000,123.000000,4.107692,176.00000,...,4.757143,81.122321,68.0,32.2,150.0,18.0,3.73,3.7,5.000000,1.006000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,44083,198330.0,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,4.107692,12.00000,...,3.900000,88.600000,144.0,14.0,26.4,14.7,4.18,10.4,5.905405,1.015864
125,44154,174245.0,-3.000000,24.000000,1.80000,48.000000,7.290000,103.000000,4.200000,25.00000,...,5.000000,90.000000,84.0,16.1,37.7,15.0,4.23,12.2,5.000000,1.017000
126,44212,163189.0,-7.000000,20.000000,2.50000,40.000000,7.280000,165.000000,3.100000,31.00000,...,1.700000,84.000000,71.0,22.4,65.9,21.2,2.68,8.8,5.000000,1.020000
127,44222,192189.0,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,4.107692,166.88764,...,5.100000,59.000000,245.0,13.2,24.5,14.5,4.05,9.9,5.000000,1.016000


In [9]:
diag = pd.read_csv('mimic-iii-clinical-database-demo-1.4/DIAGNOSES_ICD.csv')
diag

,row_id,subject_id,hadm_id,seq_num,icd9_code
0,112344,10006,142345,1,99591
1,112345,10006,142345,2,99662
2,112346,10006,142345,3,5672
3,112347,10006,142345,4,40391
4,112348,10006,142345,5,42731
...,...,...,...,...,...
1756,397673,44228,103379,7,1975
1757,397674,44228,103379,8,45182
1758,397675,44228,103379,9,99592
1759,397676,44228,103379,10,2449


In [10]:
diag = diag[diag['seq_num'] == 1]
diag = diag[['hadm_id', 'icd9_code']]
diag

,hadm_id,icd9_code
0,142345,99591
21,105331,570
27,165520,0389
36,199207,81201
50,177759,0389
...,...,...
1700,198330,4239
1706,174245,0389
1715,163189,03811
1739,192189,42781


In [11]:
admissions = pd.read_csv('mimic-iii-clinical-database-demo-1.4/ADMISSIONS.csv')
admissions = admissions[['subject_id', 'hadm_id']]
admissions

,subject_id,hadm_id
0,10006,142345
1,10011,105331
2,10013,165520
3,10017,199207
4,10019,177759
...,...,...
124,44083,198330
125,44154,174245
126,44212,163189
127,44222,192189


In [12]:
gender = pd.read_csv('mimic-iii-clinical-database-demo-1.4/PATIENTS.csv')
gender = gender[['subject_id', 'gender']]
gender

,subject_id,gender
0,10006,F
1,10011,F
2,10013,F
3,10017,F
4,10019,M
...,...,...
95,44083,M
96,44154,M
97,44212,F
98,44222,M


In [19]:
final_df = pd.merge(
    gender, 
    admissions, 
    on='subject_id',  # Column to join on
    how='left'        # Keep all rows from df1
)

final_df

,subject_id,gender,hadm_id
0,10006,F,142345
1,10011,F,105331
2,10013,F,165520
3,10017,F,199207
4,10019,M,177759
...,...,...,...
124,44083,M,198330
125,44154,M,174245
126,44212,F,163189
127,44222,M,192189


In [17]:
diag

,hadm_id,icd9_code
0,142345,99591
21,105331,570
27,165520,0389
36,199207,81201
50,177759,0389
...,...,...
1700,198330,4239
1706,174245,0389
1715,163189,03811
1739,192189,42781


In [20]:
final_df = pd.merge(
    final_df, 
    diag, 
    on='hadm_id', 
    how='left'     
)

final_df

,subject_id,gender,hadm_id,icd9_code
0,10006,F,142345,99591
1,10011,F,105331,570
2,10013,F,165520,0389
3,10017,F,199207,81201
4,10019,M,177759,0389
...,...,...,...,...
124,44083,M,198330,4239
125,44154,M,174245,0389
126,44212,F,163189,03811
127,44222,M,192189,42781


In [22]:
pivot_df.drop('subject_id', axis=1, inplace=True)

In [23]:
final_df = pd.merge(
    final_df, 
    pivot_df, 
    on='hadm_id',  
    how='left'     
)

final_df

,subject_id,gender,hadm_id,icd9_code,50802,50804,50813,50818,50820,50821,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,10006,F,142345,99591,-0.703297,25.681319,4.40000,44.197802,7.362632,167.428571,...,5.000000,86.600000,116.0,21.2,32.7,20.0,4.46,7.8,8.000000,1.012000
1,10011,F,105331,570,0.000000,23.000000,2.50297,31.000000,7.470000,119.000000,...,7.000000,85.000000,299.0,35.4,66.0,18.9,4.07,10.6,6.500000,1.020000
2,10013,F,165520,0389,2.000000,32.000000,2.50297,63.000000,7.300000,60.000000,...,7.000000,74.000000,220.0,14.6,30.4,14.2,3.48,16.2,5.000000,1.010000
3,10017,F,199207,81201,5.000000,32.000000,2.50297,47.000000,7.420000,138.000000,...,6.000000,75.000000,399.0,11.3,19.6,12.6,4.84,30.5,5.000000,1.024000
4,10019,M,177759,0389,-9.000000,19.000000,6.80000,46.000000,7.200000,123.000000,...,4.757143,81.122321,68.0,32.2,150.0,18.0,3.73,3.7,5.000000,1.006000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,44083,M,198330,4239,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,...,3.900000,88.600000,144.0,14.0,26.4,14.7,4.18,10.4,5.905405,1.015864
125,44154,M,174245,0389,-3.000000,24.000000,1.80000,48.000000,7.290000,103.000000,...,5.000000,90.000000,84.0,16.1,37.7,15.0,4.23,12.2,5.000000,1.017000
126,44212,F,163189,03811,-7.000000,20.000000,2.50000,40.000000,7.280000,165.000000,...,1.700000,84.000000,71.0,22.4,65.9,21.2,2.68,8.8,5.000000,1.020000
127,44222,M,192189,42781,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,...,5.100000,59.000000,245.0,13.2,24.5,14.5,4.05,9.9,5.000000,1.016000


In [24]:
final_df['icd9_code'] = final_df['icd9_code'].astype(str)

In [25]:
final_df

,subject_id,gender,hadm_id,icd9_code,50802,50804,50813,50818,50820,50821,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,10006,F,142345,99591,-0.703297,25.681319,4.40000,44.197802,7.362632,167.428571,...,5.000000,86.600000,116.0,21.2,32.7,20.0,4.46,7.8,8.000000,1.012000
1,10011,F,105331,570,0.000000,23.000000,2.50297,31.000000,7.470000,119.000000,...,7.000000,85.000000,299.0,35.4,66.0,18.9,4.07,10.6,6.500000,1.020000
2,10013,F,165520,0389,2.000000,32.000000,2.50297,63.000000,7.300000,60.000000,...,7.000000,74.000000,220.0,14.6,30.4,14.2,3.48,16.2,5.000000,1.010000
3,10017,F,199207,81201,5.000000,32.000000,2.50297,47.000000,7.420000,138.000000,...,6.000000,75.000000,399.0,11.3,19.6,12.6,4.84,30.5,5.000000,1.024000
4,10019,M,177759,0389,-9.000000,19.000000,6.80000,46.000000,7.200000,123.000000,...,4.757143,81.122321,68.0,32.2,150.0,18.0,3.73,3.7,5.000000,1.006000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,44083,M,198330,4239,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,...,3.900000,88.600000,144.0,14.0,26.4,14.7,4.18,10.4,5.905405,1.015864
125,44154,M,174245,0389,-3.000000,24.000000,1.80000,48.000000,7.290000,103.000000,...,5.000000,90.000000,84.0,16.1,37.7,15.0,4.23,12.2,5.000000,1.017000
126,44212,F,163189,03811,-7.000000,20.000000,2.50000,40.000000,7.280000,165.000000,...,1.700000,84.000000,71.0,22.4,65.9,21.2,2.68,8.8,5.000000,1.020000
127,44222,M,192189,42781,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,...,5.100000,59.000000,245.0,13.2,24.5,14.5,4.05,9.9,5.000000,1.016000


In [26]:
from sklearn.preprocessing import LabelEncoder

gender_encoder = LabelEncoder()

final_df['gender'] = gender_encoder.fit_transform(final_df['gender'])

In [27]:
final_df[['icd9_code', 'gender']]

,icd9_code,gender
0,99591,0
1,570,0
2,0389,0
3,81201,0
4,0389,1
...,...,...
124,4239,1
125,0389,1
126,03811,0
127,42781,1


In [28]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
icd_encoded = encoder.fit_transform(final_df[['icd9_code']])
icd_encoded_df = pd.DataFrame(
    icd_encoded,
    columns=encoder.get_feature_names_out(['icd9_code'])
)

icd_encoded_df

,icd9_code_00845,icd9_code_0380,icd9_code_03811,icd9_code_03812,icd9_code_0383,icd9_code_03842,icd9_code_03849,icd9_code_0388,icd9_code_0389,icd9_code_0543,...,icd9_code_85206,icd9_code_85221,icd9_code_85225,icd9_code_86404,icd9_code_8830,icd9_code_9693,icd9_code_99591,icd9_code_99664,icd9_code_99667,icd9_code_99731
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
final_df

,subject_id,gender,hadm_id,icd9_code,50802,50804,50813,50818,50820,50821,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,10006,0,142345,99591,-0.703297,25.681319,4.40000,44.197802,7.362632,167.428571,...,5.000000,86.600000,116.0,21.2,32.7,20.0,4.46,7.8,8.000000,1.012000
1,10011,0,105331,570,0.000000,23.000000,2.50297,31.000000,7.470000,119.000000,...,7.000000,85.000000,299.0,35.4,66.0,18.9,4.07,10.6,6.500000,1.020000
2,10013,0,165520,0389,2.000000,32.000000,2.50297,63.000000,7.300000,60.000000,...,7.000000,74.000000,220.0,14.6,30.4,14.2,3.48,16.2,5.000000,1.010000
3,10017,0,199207,81201,5.000000,32.000000,2.50297,47.000000,7.420000,138.000000,...,6.000000,75.000000,399.0,11.3,19.6,12.6,4.84,30.5,5.000000,1.024000
4,10019,1,177759,0389,-9.000000,19.000000,6.80000,46.000000,7.200000,123.000000,...,4.757143,81.122321,68.0,32.2,150.0,18.0,3.73,3.7,5.000000,1.006000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,44083,1,198330,4239,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,...,3.900000,88.600000,144.0,14.0,26.4,14.7,4.18,10.4,5.905405,1.015864
125,44154,1,174245,0389,-3.000000,24.000000,1.80000,48.000000,7.290000,103.000000,...,5.000000,90.000000,84.0,16.1,37.7,15.0,4.23,12.2,5.000000,1.017000
126,44212,0,163189,03811,-7.000000,20.000000,2.50000,40.000000,7.280000,165.000000,...,1.700000,84.000000,71.0,22.4,65.9,21.2,2.68,8.8,5.000000,1.020000
127,44222,1,192189,42781,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,...,5.100000,59.000000,245.0,13.2,24.5,14.5,4.05,9.9,5.000000,1.016000


In [30]:
X = final_df.drop(['hadm_id', 'icd9_code', 'subject_id'], axis=1)
X

,gender,50802,50804,50813,50818,50820,50821,50822,50861,50862,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,0,-0.703297,25.681319,4.40000,44.197802,7.362632,167.428571,4.107692,9.00000,3.400000,...,5.000000,86.600000,116.0,21.2,32.7,20.0,4.46,7.8,8.000000,1.012000
1,0,0.000000,23.000000,2.50297,31.000000,7.470000,119.000000,4.107692,254.00000,2.600000,...,7.000000,85.000000,299.0,35.4,66.0,18.9,4.07,10.6,6.500000,1.020000
2,0,2.000000,32.000000,2.50297,63.000000,7.300000,60.000000,4.107692,29.00000,3.101235,...,7.000000,74.000000,220.0,14.6,30.4,14.2,3.48,16.2,5.000000,1.010000
3,0,5.000000,32.000000,2.50297,47.000000,7.420000,138.000000,4.107692,12.00000,2.800000,...,6.000000,75.000000,399.0,11.3,19.6,12.6,4.84,30.5,5.000000,1.024000
4,1,-9.000000,19.000000,6.80000,46.000000,7.200000,123.000000,4.107692,176.00000,3.200000,...,4.757143,81.122321,68.0,32.2,150.0,18.0,3.73,3.7,5.000000,1.006000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,1,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,4.107692,12.00000,3.700000,...,3.900000,88.600000,144.0,14.0,26.4,14.7,4.18,10.4,5.905405,1.015864
125,1,-3.000000,24.000000,1.80000,48.000000,7.290000,103.000000,4.200000,25.00000,3.101235,...,5.000000,90.000000,84.0,16.1,37.7,15.0,4.23,12.2,5.000000,1.017000
126,0,-7.000000,20.000000,2.50000,40.000000,7.280000,165.000000,3.100000,31.00000,2.900000,...,1.700000,84.000000,71.0,22.4,65.9,21.2,2.68,8.8,5.000000,1.020000
127,1,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,4.107692,166.88764,3.101235,...,5.100000,59.000000,245.0,13.2,24.5,14.5,4.05,9.9,5.000000,1.016000


In [31]:
y = icd_encoded_df
y

,icd9_code_00845,icd9_code_0380,icd9_code_03811,icd9_code_03812,icd9_code_0383,icd9_code_03842,icd9_code_03849,icd9_code_0388,icd9_code_0389,icd9_code_0543,...,icd9_code_85206,icd9_code_85221,icd9_code_85225,icd9_code_86404,icd9_code_8830,icd9_code_9693,icd9_code_99591,icd9_code_99664,icd9_code_99667,icd9_code_99731
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
y_labels = y.idxmax(axis=1)

In [33]:
X

,gender,50802,50804,50813,50818,50820,50821,50822,50861,50862,...,51254,51256,51265,51274,51275,51277,51279,51301,51491,51498
0,0,-0.703297,25.681319,4.40000,44.197802,7.362632,167.428571,4.107692,9.00000,3.400000,...,5.000000,86.600000,116.0,21.2,32.7,20.0,4.46,7.8,8.000000,1.012000
1,0,0.000000,23.000000,2.50297,31.000000,7.470000,119.000000,4.107692,254.00000,2.600000,...,7.000000,85.000000,299.0,35.4,66.0,18.9,4.07,10.6,6.500000,1.020000
2,0,2.000000,32.000000,2.50297,63.000000,7.300000,60.000000,4.107692,29.00000,3.101235,...,7.000000,74.000000,220.0,14.6,30.4,14.2,3.48,16.2,5.000000,1.010000
3,0,5.000000,32.000000,2.50297,47.000000,7.420000,138.000000,4.107692,12.00000,2.800000,...,6.000000,75.000000,399.0,11.3,19.6,12.6,4.84,30.5,5.000000,1.024000
4,1,-9.000000,19.000000,6.80000,46.000000,7.200000,123.000000,4.107692,176.00000,3.200000,...,4.757143,81.122321,68.0,32.2,150.0,18.0,3.73,3.7,5.000000,1.006000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,1,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,4.107692,12.00000,3.700000,...,3.900000,88.600000,144.0,14.0,26.4,14.7,4.18,10.4,5.905405,1.015864
125,1,-3.000000,24.000000,1.80000,48.000000,7.290000,103.000000,4.200000,25.00000,3.101235,...,5.000000,90.000000,84.0,16.1,37.7,15.0,4.23,12.2,5.000000,1.017000
126,0,-7.000000,20.000000,2.50000,40.000000,7.280000,165.000000,3.100000,31.00000,2.900000,...,1.700000,84.000000,71.0,22.4,65.9,21.2,2.68,8.8,5.000000,1.020000
127,1,-0.703297,25.681319,2.50297,44.197802,7.362632,167.428571,4.107692,166.88764,3.101235,...,5.100000,59.000000,245.0,13.2,24.5,14.5,4.05,9.9,5.000000,1.016000


In [35]:
X.columns = X.columns.astype(str)

In [36]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler() 
X_scaled = scaler.fit_transform(X)

X_scaled

array([[-1.08923858e+00, -2.33233803e-17,  0.00000000e+00, ...,
        -5.47132378e-01,  2.27080165e+00, -5.36071556e-01],
       [-1.08923858e+00,  1.47747399e-01, -4.63569372e-01, ...,
        -2.48290378e-01,  6.44614662e-01,  5.73911901e-01],
       [-1.08923858e+00,  5.67904067e-01,  1.09242782e+00, ...,
         3.49393624e-01, -9.81572326e-01, -8.13567420e-01],
       ...,
       [-1.08923858e+00, -1.32280094e+00, -9.82235105e-01, ...,
        -4.40403092e-01, -9.81572326e-01,  5.73911901e-01],
       [ 9.18072515e-01, -2.33233803e-17,  0.00000000e+00, ...,
        -3.23000878e-01, -9.81572326e-01,  1.89201726e-02],
       [-1.08923858e+00, -1.53287927e+00, -1.32801226e+00, ...,
         7.54964910e-01,  6.44614662e-01, -1.50730708e+00]])

In [37]:
y

,icd9_code_00845,icd9_code_0380,icd9_code_03811,icd9_code_03812,icd9_code_0383,icd9_code_03842,icd9_code_03849,icd9_code_0388,icd9_code_0389,icd9_code_0543,...,icd9_code_85206,icd9_code_85221,icd9_code_85225,icd9_code_86404,icd9_code_8830,icd9_code_9693,icd9_code_99591,icd9_code_99664,icd9_code_99667,icd9_code_99731
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
125,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
126,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
127,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
import numpy as np
y_class = np.argmax(y.values, axis=1)
y_class

array([79, 58,  8, 70,  8, 39, 35,  1, 71, 22, 40,  8, 81, 26, 29, 48, 17,
        4, 17,  8, 59, 59, 35, 66, 12, 76, 25, 57, 44, 24,  8, 29,  8, 39,
       78, 29, 21,  8, 67, 14, 20, 64,  9, 35, 33, 35, 51, 44, 61, 53, 58,
       72, 49, 58, 69, 56, 10, 44, 44, 62, 64, 73, 65, 16, 18, 19, 37, 29,
       52, 27, 61,  7, 48, 48, 48, 82,  8,  3,  0, 48, 44,  8,  8, 80,  6,
        8,  8,  6, 54,  8,  8, 68, 36, 50, 28, 60, 60, 29, 15, 45, 59, 77,
       46, 64, 36, 55, 75, 23, 41, 43, 63, 13, 38, 74, 12, 28, 42, 48, 31,
       11, 44, 30, 12, 47, 32,  8,  2, 34,  5])

In [51]:
import xgboost as xgb

model = xgb.XGBClassifier(
    objective='multi:softmax', 
    num_class=len(y.columns),  
    eval_metric='mlogloss',    
    n_estimators=100,
    random_state=42
)

model.fit(X_scaled, y_class)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=83, num_parallel_tree=None, ...)

In [54]:
y_class

array([79, 58,  8, 70,  8, 39, 35,  1, 71, 22, 40,  8, 81, 26, 29, 48, 17,
        4, 17,  8, 59, 59, 35, 66, 12, 76, 25, 57, 44, 24,  8, 29,  8, 39,
       78, 29, 21,  8, 67, 14, 20, 64,  9, 35, 33, 35, 51, 44, 61, 53, 58,
       72, 49, 58, 69, 56, 10, 44, 44, 62, 64, 73, 65, 16, 18, 19, 37, 29,
       52, 27, 61,  7, 48, 48, 48, 82,  8,  3,  0, 48, 44,  8,  8, 80,  6,
        8,  8,  6, 54,  8,  8, 68, 36, 50, 28, 60, 60, 29, 15, 45, 59, 77,
       46, 64, 36, 55, 75, 23, 41, 43, 63, 13, 38, 74, 12, 28, 42, 48, 31,
       11, 44, 30, 12, 47, 32,  8,  2, 34,  5])

In [56]:
y_pred = model.predict(X_scaled)

In [57]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_class, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 1.00


In [62]:
import lightgbm as lgb
from sklearn.metrics import f1_score

lgb_params = {
    'objective': 'multiclass',
    'num_class': len(y.columns),
    'metric': 'multi_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'verbose': -1
}

train_data = lgb.Dataset(X_scaled, label=y_class)

lgb_model = lgb.train(
    lgb_params,
    train_data,
    num_boost_round=200,
    valid_sets=[train_data],
    callbacks=[lgb.log_evaluation(50)]
)

y_pred_lgb = lgb_model.predict(X_scaled)

[50]	training's multi_logloss: 0.12968
[100]	training's multi_logloss: 0.00530691
[150]	training's multi_logloss: 0.000745132
[200]	training's multi_logloss: 0.000438453


In [63]:
y_pred_lgb

array([[1.23842739e-06, 1.34908664e-06, 1.34660023e-06, ...,
        4.53691378e-06, 2.24072875e-06, 1.13843746e-06],
       [1.01372285e-06, 6.57121685e-07, 3.08405768e-05, ...,
        6.17424131e-07, 2.36879221e-05, 6.82187659e-07],
       [2.52264749e-06, 6.68182816e-06, 2.02394909e-06, ...,
        3.57446878e-06, 6.30323853e-06, 2.34205228e-06],
       ...,
       [4.35142066e-05, 1.02766105e-06, 9.99255149e-01, ...,
        2.95568500e-07, 1.15958666e-05, 1.54384405e-06],
       [3.00968506e-06, 1.50874437e-06, 1.71146300e-06, ...,
        1.79832618e-05, 6.05383486e-07, 2.77254795e-06],
       [1.09051064e-06, 2.31733040e-07, 7.53430332e-07, ...,
        1.11448449e-07, 2.09903600e-06, 6.36926142e-07]])

In [64]:
y_pred_lgb_class = np.argmax(y_pred_lgb, axis=1)
print("LightGBM F1-Score:", f1_score(y_class, y_pred_lgb_class, average='weighted'))

LightGBM F1-Score: 1.0
